# Download dataset

In [ ]:
from distutils.dir_util import copy_tree
fromDir = "/kaggle/input/resnext-3-caps"
desDir = "./"
copy_tree(fromDir, desDir)

In [ ]:
import sys
sys.setrecursionlimit(15000)
import torch
import torch.nn.functional as F
from torch import nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torchvision.models as models

In [ ]:
NO_CAPS=3

In [ ]:
class StatsNet(nn.Module):
    def __init__(self):
        super(StatsNet, self).__init__()

    def forward(self, x):
        x = x.view(x.data.shape[0], x.data.shape[1], x.data.shape[2]*x.data.shape[3])

        mean = torch.mean(x, 2)
        std = torch.std(x, 2)

        return torch.stack((mean, std), dim=1)

In [ ]:
class View(nn.Module):
    def __init__(self, *shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, input):
        return input.view(self.shape)

In [ ]:
class VggExtractor(nn.Module):
    def __init__(self, train=False):
        super(VggExtractor, self).__init__()

#         self.vgg_1 = self.Vgg(models.resnext50_32x4d(pretrained=True), 0, 3)
#         self.vgg_1 = self.Vgg(torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d', pretrained=True), 0, 2)
#         self.vgg_1 = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d', pretrained=True)
        self.vgg_1 = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d', pretrained=True)
        modules = list(self.vgg_1.children())
        self.vgg_1 = torch.nn.Sequential(*(list(modules)[:5]))
        
        if train:
            self.vgg_1.train(mode=True)
            self.freeze_gradient()
        else:
            self.vgg_1.eval()

    def Vgg(self, vgg, begin, end):
        features = nn.Sequential(*list(vgg.features.children())[begin:(end+1)])
        return features

    def freeze_gradient(self, begin=0, end=9):
        for i in range(begin, end+1):
            self.vgg_1[i].requires_grad = False

    def forward(self, input):
        return self.vgg_1(input)

In [ ]:
# vgg_ext = VggExtractor()
# vgg_ext

In [ ]:
# nn.Sequential(*list(vgg.features.children())[begin:(end+1)])
# nn.Sequential(list(vgg_ext.children())[0])
# a, *b = nn.Sequential(list(vgg_ext.children())[0])

In [ ]:
encoder = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d', pretrained=True)
modules = list(encoder.children())
encoder = torch.nn.Sequential(*(list(modules)[:5]))
encoder

In [ ]:
!pip install torchsummary
from torchsummary import summary

summary(encoder.cuda(0), (3,256,256))

In [ ]:
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()

        self.capsules = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(256, 64, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.Conv2d(64, 16, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(16),
                nn.ReLU(),
                StatsNet(),

                nn.Conv1d(2, 8, kernel_size=5, stride=2, padding=2),
                nn.BatchNorm1d(8),
                nn.Conv1d(8, 1, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm1d(1),
                View(-1, 8),
                )
                for _ in range(NO_CAPS)]
        )

    def squash(self, tensor, dim):
        squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1 + squared_norm)
        return scale * tensor / (torch.sqrt(squared_norm))

    def forward(self, x):
        # outputs = [capsule(x.detach()) for capsule in self.capsules]
        # outputs = [capsule(x.clone()) for capsule in self.capsules]
        outputs = [capsule(x) for capsule in self.capsules]
        output = torch.stack(outputs, dim=-1)

        return self.squash(output, dim=-1)

In [ ]:
class RoutingLayer(nn.Module):
    def __init__(self, gpu_id, num_input_capsules, num_output_capsules, data_in, data_out, num_iterations):
        super(RoutingLayer, self).__init__()

        self.gpu_id = gpu_id
        self.num_iterations = num_iterations
        self.route_weights = nn.Parameter(torch.randn(num_output_capsules, num_input_capsules, data_out, data_in))


    def squash(self, tensor, dim):
        squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1 + squared_norm)
        return scale * tensor / (torch.sqrt(squared_norm))

    def forward(self, x, random, dropout):
        # x[b, data, in_caps]

        x = x.transpose(2, 1)
        # x[b, in_caps, data]

        if random:
            noise = Variable(0.01*torch.randn(*self.route_weights.size()))
            if self.gpu_id >= 0:
                noise = noise.cuda(self.gpu_id)
            route_weights = self.route_weights + noise
        else:
            route_weights = self.route_weights

        priors = route_weights[:, None, :, :, :] @ x[None, :, :, :, None]

        # route_weights [out_caps , 1 , in_caps , data_out , data_in]
        # x             [   1     , b , in_caps , data_in ,    1    ]
        # priors        [out_caps , b , in_caps , data_out,    1    ]

        priors = priors.transpose(1, 0)
        # priors[b, out_caps, in_caps, data_out, 1]

        if dropout > 0.0:
            drop = Variable(torch.FloatTensor(*priors.size()).bernoulli(1.0- dropout))
            if self.gpu_id >= 0:
                drop = drop.cuda(self.gpu_id)
            priors = priors * drop
            

        logits = Variable(torch.zeros(*priors.size()))
        # logits[b, out_caps, in_caps, data_out, 1]

        if self.gpu_id >= 0:
            logits = logits.cuda(self.gpu_id)

        num_iterations = self.num_iterations

        for i in range(num_iterations):
            probs = F.softmax(logits, dim=2)
            outputs = self.squash((probs * priors).sum(dim=2, keepdim=True), dim=3)

            if i != self.num_iterations - 1:
                delta_logits = priors * outputs
                logits = logits + delta_logits

        # outputs[b, out_caps, 1, data_out, 1]
        outputs = outputs.squeeze()

        if len(outputs.shape) == 3:
            outputs = outputs.transpose(2, 1).contiguous() 
        else:
            outputs = outputs.unsqueeze_(dim=0).transpose(2, 1).contiguous()
        # outputs[b, data_out, out_caps]

        return outputs

In [ ]:
class CapsuleNet(nn.Module):
    def __init__(self, num_class, gpu_id):
        super(CapsuleNet, self).__init__()

        self.num_class = num_class
        self.fea_ext = FeatureExtractor()
        self.fea_ext.apply(self.weights_init)

        self.routing_stats = RoutingLayer(gpu_id=gpu_id, num_input_capsules=NO_CAPS, num_output_capsules=num_class, data_in=8, data_out=4, num_iterations=2)

    def weights_init(self, m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            m.weight.data.normal_(0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            m.weight.data.normal_(1.0, 0.02)
            m.bias.data.fill_(0)

    def forward(self, x, random=False, dropout=0.0):

        z = self.fea_ext(x)
        z = self.routing_stats(z, random, dropout=dropout)
        # z[b, data, out_caps]

        # classes = F.softmax(z, dim=-1)

        # class_ = classes.detach()
        # class_ = class_.mean(dim=1)

        # return classes, class_

        classes = F.softmax(z, dim=-1)
        class_ = classes.detach()
        class_ = class_.mean(dim=1)

        return z, class_

In [ ]:
class CapsuleLoss(nn.Module):
    def __init__(self, gpu_id):
        super(CapsuleLoss, self).__init__()
        self.cross_entropy_loss = nn.CrossEntropyLoss()

        if gpu_id >= 0:
            self.cross_entropy_loss.cuda(gpu_id)

    def forward(self, classes, labels):
        # abc = F.tanh(classes[:,0,0])
        # labels[labels == 0] = -1
        loss_t = self.cross_entropy_loss(classes[:,0,:], labels)

        for i in range(classes.size(1) - 1):
            loss_t = loss_t + self.cross_entropy_loss(classes[:,i+1,:], labels)

        return loss_t

# Thêm thư viện

In [ ]:
import sys
sys.setrecursionlimit(15000)
import os
import random
import torch
import torch.backends.cudnn as cudnn
import numpy as np
from torch.autograd import Variable
from torch.optim import Adam
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn import metrics

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Cài đặt biến

In [ ]:
manualSeed = None
gpu_id = 0
#choose a epochs to resume from (0 to train from scratch)
resume = 19
#folder to output model checkpoints
outf = "./"
#beta1 for adam
beta1 = 0.9
#learning rate
lr = 0.0005
#path to root dataset
dataset = "../input/ffpps/FFPPx100"
train_set = "train"
val_set = "valid"
#number of data loading workers
workers = 0
batchSize = 32 #default = 32
#the height / width of the input image to network
imageSize = 256
#number of epochs to train for
niter = 35
#disable randomness for routing matrix
disable_random = False
#dropout percentage
dropout= 0.05

## Cài đặt seed

In [ ]:
if manualSeed is None:
    manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

## Kiểm tra GPU

In [ ]:
if gpu_id >= 0:
    torch.cuda.manual_seed_all(manualSeed)
    cudnn.benchmark = True

## Khôi phục checkpoint

In [ ]:
if resume > 0:
    text_writer = open(os.path.join(outf, 'train.csv'), 'a')
else:
    text_writer = open(os.path.join(outf, 'train.csv'), 'w')

## Cài VGG, Capsnet và loss

In [ ]:
vgg_ext = VggExtractor()
capnet = CapsuleNet(2, gpu_id)
capsule_loss = CapsuleLoss(gpu_id)

## Cài đặt optimizer

In [ ]:
optimizer = Adam(capnet.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
if resume > 0:
    capnet.load_state_dict(torch.load(os.path.join(outf,'capsule_' + str(resume) + '.pt')))
    capnet.train(mode=True)
    optimizer.load_state_dict(torch.load(os.path.join(outf,'optim_' + str(resume) + '.pt')))

    if gpu_id >= 0:
        for state in optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor):
                    state[k] = v.cuda(gpu_id)


In [ ]:
if gpu_id >= 0:
    capnet.cuda(gpu_id)
    vgg_ext.cuda(gpu_id)
    capsule_loss.cuda(gpu_id)

In [ ]:
transform_fwd = transforms.Compose([
    transforms.Resize((imageSize, imageSize)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Traning

## Setup training set

In [ ]:
dataset_train = dset.ImageFolder(root=os.path.join(dataset, train_set), transform=transform_fwd)
assert dataset_train
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batchSize, shuffle=True, num_workers=int(workers))

## Setup validation set

In [ ]:
dataset_val = dset.ImageFolder(root=os.path.join(dataset, val_set), transform=transform_fwd)
assert dataset_val
dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=batchSize, shuffle=False, num_workers=int(workers))

In [ ]:
for epoch in range(resume+1, niter+1):
    count = 0
    loss_train = 0
    loss_test = 0

    tol_label = np.array([], dtype=np.float)
    tol_pred = np.array([], dtype=np.float)

    for img_data, labels_data in tqdm(dataloader_train):

        labels_data[labels_data > 1] = 1
        img_label = labels_data.numpy().astype(np.float)
        optimizer.zero_grad()

        if gpu_id >= 0:
            img_data = img_data.cuda(gpu_id)
            labels_data = labels_data.cuda(gpu_id)

        input_v = Variable(img_data)
        x = vgg_ext(input_v)
        classes, class_ = capnet(x, random=random, dropout=dropout)

        loss_dis = capsule_loss(classes, Variable(labels_data, requires_grad=False))
        loss_dis_data = loss_dis.item()

        loss_dis.backward()
        optimizer.step()

        output_dis = class_.data.cpu().numpy()
        output_pred = np.zeros((output_dis.shape[0]), dtype=np.float)

        for i in range(output_dis.shape[0]):
            if output_dis[i,1] >= output_dis[i,0]:
                output_pred[i] = 1.0
            else:
                output_pred[i] = 0.0

        tol_label = np.concatenate((tol_label, img_label))
        tol_pred = np.concatenate((tol_pred, output_pred))

        loss_train += loss_dis_data
        count += 1


    acc_train = metrics.accuracy_score(tol_label, tol_pred)
    loss_train /= count

    ########################################################################

    # do checkpointing & validation
    torch.save(capnet.state_dict(), os.path.join(outf, 'capsule_%d.pt' % epoch))
    torch.save(optimizer.state_dict(), os.path.join(outf, 'optim_%d.pt' % epoch))

    capnet.eval()

    tol_label = np.array([], dtype=np.float)
    tol_pred = np.array([], dtype=np.float)

    count = 0

    for img_data, labels_data in dataloader_val:

        labels_data[labels_data > 1] = 1
        img_label = labels_data.numpy().astype(np.float)

        if gpu_id >= 0:
            img_data = img_data.cuda(gpu_id)
            labels_data = labels_data.cuda(gpu_id)

        input_v = Variable(img_data)

        x = vgg_ext(input_v)
        classes, class_ = capnet(x, random=False)

        loss_dis = capsule_loss(classes, Variable(labels_data, requires_grad=False))
        loss_dis_data = loss_dis.item()
        output_dis = class_.data.cpu().numpy()

        output_pred = np.zeros((output_dis.shape[0]), dtype=np.float)

        for i in range(output_dis.shape[0]):
            if output_dis[i,1] >= output_dis[i,0]:
                output_pred[i] = 1.0
            else:
                output_pred[i] = 0.0

        tol_label = np.concatenate((tol_label, img_label))
        tol_pred = np.concatenate((tol_pred, output_pred))

        loss_test += loss_dis_data
        count += 1

    acc_test = metrics.accuracy_score(tol_label, tol_pred)
    loss_test /= count

    print('[Epoch %d] Train loss: %.4f   acc: %.2f | Test loss: %.4f  acc: %.2f'
    % (epoch, loss_train, acc_train*100, loss_test, acc_test*100))

    text_writer.write('%d,%.4f,%.2f,%.4f,%.2f\n'
    % (epoch, loss_train, acc_train*100, loss_test, acc_test*100))

    text_writer.flush()
    capnet.train(mode=True)

text_writer.close()

In [ ]:
# !pip install torchsummary
# from torchsummary import summary

# encoder = torch.hub.load('pytorch/vision:v0.10.0', 'resnext50_32x4d', pretrained=True)
# modules = list(encoder.children())
# encoder = torch.nn.Sequential(*(list(modules)[:5]))

# summary(encoder.cuda(0), (3,256,256))